# Solving Einstein equation to get Kottler solution

This Jupyter/SageMath worksheet is relative to the lectures
[Geometry and physics of black holes](http://luth.obspm.fr/~luthier/gourgoulhon/bh16/)
  
These computations are based on [SageManifolds](http://sagemanifolds.obspm.fr) (v0.9)

Click [here](https://raw.githubusercontent.com/egourgoulhon/BHLectures/master/sage/Kottler_solution.ipynb) to download the worksheet file (ipynb format). To run it, you must start SageMath with the Jupyter notebook, with the command `sage -n jupyter`

First we set up the notebook to display mathematical objects using LaTeX formatting:

In [1]:
%display latex

## Spacetime

We declare the spacetime manifold $M$:

In [2]:
M = Manifold(4, 'M')
print(M)

4-dimensional differentiable manifold M


We declare the chart of spherical coordinates $(t,r,\theta,\phi)$:

In [3]:
X.<t,r,th,ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
X

Chart (M, (t, r, th, ph))

The static and spherically symmetric metric ansatz, with the unknown functions $A(r)$ and $B(r)$:

In [4]:
g = M.lorentzian_metric('g')
A = function('A')
B = function('B')
g[0,0] = -A(r)
g[1,1] = B(r)
g[2,2] = r^2
g[3,3] = (r*sin(th))^2
g.display()

g = -A(r) dt*dt + B(r) dr*dr + r^2 dth*dth + r^2*sin(th)^2 dph*dph

The Christoffel symbols of $g$, with respect to the default chart:

In [5]:
g.christoffel_symbols_display()

Gam^t_t,r = 1/2*d(A)/dr/A(r) 
Gam^r_t,t = 1/2*d(A)/dr/B(r) 
Gam^r_r,r = 1/2*d(B)/dr/B(r) 
Gam^r_th,th = -r/B(r) 
Gam^r_ph,ph = -r*sin(th)^2/B(r) 
Gam^th_r,th = 1/r 
Gam^th_ph,ph = -cos(th)*sin(th) 
Gam^ph_r,ph = 1/r 
Gam^ph_th,ph = cos(th)/sin(th)

## Einstein equation

The cosmological constant:

In [6]:
var('Lamb', latex_name='\Lambda')

Lamb

The Einstein equation:

In [7]:
EE = g.ricci() - 1/2*g.ricci_scalar()*g + Lamb*g
EE.set_name('E')
print(EE)

Field of symmetric bilinear forms E on the 4-dimensional differentiable manifold M


In [8]:
EE.display_comp()

E_t,t = -((Lamb*r^2 - 1)*A(r)*B(r)^2 - r*A(r)*d(B)/dr + A(r)*B(r))/(r^2*B(r)^2) 
E_r,r = ((Lamb*r^2 - 1)*A(r)*B(r) + r*d(A)/dr + A(r))/(r^2*A(r)) 
E_th,th = 1/4*(4*Lamb*r^2*A(r)^2*B(r)^2 - r^2*B(r)*(d(A)/dr)^2 + 2*r^2*A(r)*B(r)*d^2(A)/dr^2 + 2*r*A(r)*B(r)*d(A)/dr - (r^2*A(r)*d(A)/dr + 2*r*A(r)^2)*d(B)/dr)/(A(r)^2*B(r)^2) 
E_ph,ph = 1/4*(4*Lamb*r^2*A(r)^2*B(r)^2 - r^2*B(r)*(d(A)/dr)^2 + 2*r^2*A(r)*B(r)*d^2(A)/dr^2 + 2*r*A(r)*B(r)*d(A)/dr - (r^2*A(r)*d(A)/dr + 2*r*A(r)^2)*d(B)/dr)*sin(th)^2/(A(r)^2*B(r)^2)

### Simplifying and rearranging the equations

In [9]:
eq0 = EE[0,0]*r^2*B(r)^2/A(r); eq0

-(Lamb*r^2 - 1)*B(r)^2 + r*d(B)/dr - B(r)

In [10]:
eq1 = EE[1,1]*r^2*A(r); eq1

(Lamb*r^2 - 1)*A(r)*B(r) + r*d(A)/dr + A(r)

In [11]:
eq2 = EE[2,2]*4*A(r)^2*B(r)^2; eq2

4*Lamb*r^2*A(r)^2*B(r)^2 - r^2*B(r)*(d(A)/dr)^2 + 2*r^2*A(r)*B(r)*d^2(A)/dr^2 + 2*r*A(r)*B(r)*d(A)/dr - (r^2*A(r)*d(A)/dr + 2*r*A(r)^2)*d(B)/dr

In [12]:
eq3 = EE[3,3]*4*A(r)^2*B(r)^2/sin(th)^2; eq3

4*Lamb*r^2*A(r)^2*B(r)^2 - r^2*B(r)*(d(A)/dr)^2 + 2*r^2*A(r)*B(r)*d^2(A)/dr^2 + 2*r*A(r)*B(r)*d(A)/dr - (r^2*A(r)*d(A)/dr + 2*r*A(r)^2)*d(B)/dr

In [13]:
eq3 == eq2

True

## Solving the Einstein equation

The following combination of eq0 and eq1 is particularly simple:

In [14]:
eq4 = (eq0*A(r) + eq1*B(r))/r; eq4

B(r)*d(A)/dr + A(r)*d(B)/dr

The solution is $A(r)B(r)=C$, where $C$ is a constant:

In [15]:
s = desolve(eq4.expr() == 0, B(r), ivar=r)
s

_C/A(r)

In [16]:
var('alpha', latex_name=r'\alpha')
B_sol(r) = s.subs(_C=alpha); B_sol

r |--> alpha/A(r)

In [17]:
eq5 = X.function(eq1.expr().substitute_function(B, B_sol)); eq5

(Lamb*r^2 - 1)*alpha + r*d(A)/dr + A(r)

In [18]:
eq6 = X.function(eq2.expr().substitute_function(B, B_sol)); eq6

4*Lamb*alpha^2*r^2 - alpha*r^2*(d(A)/dr)^2/A(r) + 2*alpha*r^2*d^2(A)/dr^2 + 2*alpha*r*d(A)/dr + (r^2*A(r)*d(A)/dr + 2*r*A(r)^2)*alpha*d(A)/dr/A(r)^2

In [19]:
s = desolve(eq5.expr() == 0, A(r), ivar=r)
s.expand()

-1/3*Lamb*alpha*r^2 + alpha + _C/r

In [20]:
var('m')
A_sol(r) = s.subs(_C=-2*m, alpha=1).expand()
A_sol

r |--> -1/3*Lamb*r^2 - 2*m/r + 1

In [21]:
eq6.expr().substitute_function(A, A_sol).subs(alpha=1).simplify_full()

0

In [22]:
g[0,0] = -A_sol(r)
g[1,1] = 1/A_sol(r)
g.display()

g = 1/3*(Lamb*r^3 + 6*m - 3*r)/r dt*dt - 3*r/(Lamb*r^3 + 6*m - 3*r) dr*dr + r^2 dth*dth + r^2*sin(th)^2 dph*dph

In [23]:
g.christoffel_symbols_display()

Gam^t_t,r = (Lamb*r^3 - 3*m)/(Lamb*r^4 + 6*m*r - 3*r^2) 
Gam^r_t,t = 1/9*(Lamb^2*r^6 + 3*Lamb*m*r^3 - 3*Lamb*r^4 - 18*m^2 + 9*m*r)/r^3 
Gam^r_r,r = -(Lamb*r^3 - 3*m)/(Lamb*r^4 + 6*m*r - 3*r^2) 
Gam^r_th,th = 1/3*Lamb*r^3 + 2*m - r 
Gam^r_ph,ph = 1/3*(Lamb*r^3 + 6*m - 3*r)*sin(th)^2 
Gam^th_r,th = 1/r 
Gam^th_ph,ph = -cos(th)*sin(th) 
Gam^ph_r,ph = 1/r 
Gam^ph_th,ph = cos(th)/sin(th)

In [24]:
EE = g.ricci() - 1/2*g.ricci_scalar()*g + Lamb*g
EE.set_name('E')
EE.display()

E = 0

In [25]:
g.ricci_scalar().display()

r(g): M --> R
   (t, r, th, ph) |--> 4*Lamb

In [26]:
g.ricci().display()

Ric(g) = 1/3*(Lamb^2*r^3 + 6*Lamb*m - 3*Lamb*r)/r dt*dt - 3*Lamb*r/(Lamb*r^3 + 6*m - 3*r) dr*dr + Lamb*r^2 dth*dth + Lamb*r^2*sin(th)^2 dph*dph

In [27]:
g.ricci() == Lamb * g

True